<a href="https://colab.research.google.com/github/meti-94/BERT-QA/blob/main/BuboQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fuzzywuzzy

In [2]:
!git clone https://github.com/castorini/BuboQA.git
%cd BuboQA

Cloning into 'BuboQA'...
remote: Enumerating objects: 1383, done.
remote: Total 1383 (delta 0), reused 0 (delta 0), pack-reused 1383
Receiving objects: 100% (1383/1383), 361.07 KiB | 3.41 MiB/s, done.
Resolving deltas: 100% (908/908), done.
/content/BuboQA


In [3]:
!bash setup.sh 

Running fetch_dataset script

--2021-03-27 18:55:42--  https://git.uwaterloo.ca/jimmylin/BuboQA-data/raw/master/SimpleQuestions_v2.tgz
Resolving git.uwaterloo.ca (git.uwaterloo.ca)... 129.97.83.4
Connecting to git.uwaterloo.ca (git.uwaterloo.ca)|129.97.83.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 423435590 (404M) [application/x-gzip]
Saving to: ‘SimpleQuestions_v2.tgz’

SimpleQuestions_v2. 100%[===================>] 403.82M  96.2MB/s    in 4.4s    

2021-03-27 18:55:53 (90.9 MB/s) - ‘SimpleQuestions_v2.tgz’ saved [423435590/423435590]



Unzipping SimpleQuestions dataset...

tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended header keyword 'SCHILY.ino'
tar: Ignoring unknown extended header keyword 'SCHILY.nlink'
./._SimpleQuestions_v2
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended header keyword 'SCHI

In [4]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2021-03-27 19:17:14--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-03-27 19:17:14--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-03-27 19:17:14--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [5]:
!unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [6]:
import pandas as pd
from collections import Counter
from tqdm import tqdm
from sklearn.preprocessing import LabelBinarizer
from copy import copy
import numpy as np
from sklearn.linear_model import LogisticRegression

In [7]:
print('Indexing word vectors.')

embeddings_index = {}
f = open('glove.6B.300d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 400000 word vectors.


In [8]:
train = pd.read_excel('/content/drive/MyDrive/QA/train.xlsx')
valid = pd.read_excel('/content/drive/MyDrive/QA/valid.xlsx')
test = pd.read_excel('/content/drive/MyDrive/QA/test.xlsx')

In [24]:
def sentence2vector(sentence):
  vectors = []
  for word in sentence.split():
    if word in embeddings_index:
      vectors.append(embeddings_index[word])
    else:
      pass
  vectors = np.array(vectors)
  vector = np.mean(vectors, axis=0)
  return vector

def get_classes_map(dataframe):
  all_normal_rels = [eval(item)[1] for item in dataframe.normalized_triple.to_list()]
  rel2class = {}
  counter = 0
  for item in all_normal_rels:
    if item in rel2class:
      pass
    else:
      rel2class[item]=counter
      counter+=1
  # list(set(all_normal_rels))
  return rel2class
def get_frequent_rels(dataframe, count):
  all_normal_rels = [eval(item)[1] for item in dataframe.normalized_triple.to_list()]
  most_frequent_rels = [word for word, word_count in Counter(all_normal_rels).most_common(count)]
  return most_frequent_rels
def get_features(dataframe, most_frequent_rels):
  onehot_features = []
  for idx, row in tqdm(dataframe.iterrows()):
    temp_rel = eval(row.normalized_triple)[1]
    if temp_rel in most_frequent_rels:
      onehot_features.append(most_frequent_rels.index(temp_rel))
    else:
      onehot_features.append(len(most_frequent_rels))
  label_binarizer = LabelBinarizer()
  label_binarizer.fit(range(max(onehot_features)+1))
  onehot_features = label_binarizer.transform(onehot_features)
  embeddings_features = np.zeros((len(dataframe), 300), )
  for idx, row in tqdm(dataframe.iterrows()):
    vector = sentence2vector(row.Question)
    embeddings_features[idx, :] = vector
  features = np.hstack((embeddings_features, onehot_features))
  return np.nan_to_num(features)
def get_labels(dataframe, classes_map):
  labels = []
  for idx, row in tqdm(dataframe.iterrows()):
    temp_rel = eval(row.normalized_triple)[1]
    if temp_rel in classes_map:
      labels.append(classes_map[temp_rel])
    else:
      labels.append(-1)
  return labels

In [25]:
most_frequent_rels = get_frequent_rels(train, 300)
class_map = get_classes_map(train)

train_featurs = get_features(train, most_frequent_rels)
train_labels = get_labels(train, class_map)
valid_featurs = get_features(valid, most_frequent_rels)
valid_labels = get_labels(valid, class_map)
test_featurs = get_features(test, most_frequent_rels)
test_labels = get_labels(test, class_map)




9921it [00:01, 5928.47it/s]
341it [00:00, 3400.80it/s]/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
9921it [00:02, 4468.89it/s]
9921it [00:01, 6329.01it/s]
1751it [00:00, 5629.55it/s]
1751it [00:00, 4653.95it/s]
1751it [00:00, 6288.38it/s]
5003it [00:00, 5966.87it/s]
5003it [00:01, 4835.78it/s]
5003it [00:00, 6495.69it/s]


In [31]:
clf = LogisticRegression()
# print(valid_featurs.shape, train_featur)
X = np.vstack((train_featurs, valid_featurs)); y = np.array(train_labels+valid_labels)

mask = y >= 0
X = X[mask]; y=y[mask]

clf.fit(X, y)

X_test, y_test = test_featurs, np.array(test_labels)
mask = y_test >= 0
X_test = X_test[mask]; y_test=y_test[mask]


predicted_test = clf.predict(X_test)
accuracy_on_test = 100.0 * np.mean(predicted_test == y_test)
print("Accuracy on Test set: {}".format(accuracy_on_test))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Accuracy on Test set: 56.273199703043794


In [33]:
print(len(predicted_test), predicted_test[6:100], y_test[6:100])

4041 [3045  751  693 1025 3629  349 4090 2637   51 1453    1 2953 3956 1384
 1290 2446  140 1549   92  197 2347 1808   18 2681 3781  737   60  147
   60 1751  850  752 1971 2152 1496  359 1631  970  654  243   18   68
  734 2174 1350 2813 1087  166    1  850  579 3970 2540  850  194  492
 1351  318  423   18 3320   33 2871  424  197 2187  988 1596  177  351
  194 2485 2539 1155  461  197   34 1384 1797 1373  308 1479 1832   68
  228 1089   18    1 1183  563 2141  918    1  129] [1410  751  693 1025 3576  349 2423 2848   51   69    1 2953 2860  458
 4060 2446  140 3366   92  197 2347 1808   18 3237 3741  737   60 3612
   60 1751 3309 1868 2664 2478 1496  359  463  630 3751  243   18   68
  734 2174 3903 3658 1087  166    1 3387  579 3970 2430 1253  194  492
 2406  318  423 2150 3956   33  929  424  197 1480 1734 3665 1960 2059
  194 1973 2198 1155  461  197 1319 1024  808 4115  308 1692 1585   68
  228 3994   18    1 1183  563 2141 3099    1  129]
